In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data17smaller_4_2.csv to data17smaller_4_2.csv


In [0]:
!unzip "data17smaller_1.csv.zip"

Archive:  data17smaller_1.csv.zip
replace data17smaller_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
data17smaller = pd.read_csv("data17smaller_3_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})


In [0]:
!pip install pgmpy==0.1.7
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator
from pgmpy.models import BayesianModel
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination

In [0]:
from collections import defaultdict
from pgmpy.inference import VariableElimination
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    count = 0
    for index, data_point in data.iterrows():
        count = count+1
        if count %1000 == 1:
          print count
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

In [0]:
data17train, data17test = train_test_split(data17smaller,random_state=33,test_size=0.2)
hc = HillClimbSearch(data17train, scoring_method=BicScore(data17train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'unit_ratio'), ('bldgclass', 'subsidies_flag'), ('bldgclass', 'landuse'), ('residfar', 'subsidies_flag'), ('yearbuilt', 'residfar'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'violations'), ('yearbuilt', 'landuse'), ('yearbuilt', 'bldgclass'), ('violations', 'bldgclass'), ('landuse', 'residfar')]


In [0]:
best_model.fit(data17train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+---------------------+-----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-----------------------+---------------------+----------------------+----------------------+----------------------+----------------------+
| violations   | violations(0)       | violations(0)         | violations(0)        | violations(0)        | violations(0)        | violations(1)        | violations(1)        | violations(1)        | violations(1)        | violations(1)         | violations(2)       | violations(2)        | violations(2)        | violations(2)        | violations(2)        |
+--------------+---------------------+-----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-----------------------+-----------

In [0]:
best_model.fit(data17train, estimator=BayesianEstimator, prior_type='K2')
test_x = data17test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data17test.loc[:,'violations']==predicted_test['violations']).mean()

Out of sample: 0.5978935218583177


In [0]:
train_x = data17train.drop(columns=['violations'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data17train.loc[:,'violations']==predicted_train['violations']).mean()

In of sample: 0.5951084015728149


In [0]:
data17smaller = pd.read_csv("data17smaller_4_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})


In [0]:
data17train, data17test = train_test_split(data17smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data17train, scoring_method=BicScore(data17train))
best_model = hc.estimate()
print(best_model.edges())


[('bldgclass', 'unit_ratio'), ('bldgclass', 'subsidies_flag'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'vio_rt'), ('residfar', 'subsidies_flag'), ('residfar', 'landuse'), ('residfar', 'yearbuilt'), ('residfar', 'bldgclass'), ('yearbuilt', 'unit_ratio'), ('yearbuilt', 'vio_rt'), ('landuse', 'bldgclass')]


In [0]:

best_model.fit(data17train, estimator=BayesianEstimator, prior_type='K2')

train_x = data17train.drop(columns=['vio_rt'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data17train.loc[:,'vio_rt']==predicted_train['vio_rt']).mean()

test_x = data17test.drop(columns=['vio_rt'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data17test.loc[:,'vio_rt']==predicted_test['vio_rt']).mean()

In of sample: 0.3487969409472963
Out of sample: 0.34410618958303274


In [0]:
best_model.fit(data17train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('vio_rt')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+------------------------+-----------------------+---------------------+---------------------+----------------------+-----------------------+------------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+
| landuse      | landuse(0)             | landuse(0)            | landuse(0)          | landuse(0)          | landuse(0)           | landuse(1)            | landuse(1)             | landuse(1)            | landuse(1)            | landuse(1)            | landuse(2)            | landuse(2)            | landuse(2)            | landuse(2)            | landuse(2)            | landuse(3)            | landuse(3)            | landuse(3)            | landuse(3)            | landuse(3)        